In [18]:
import json
import pandas as pd
import numpy as np

In [83]:
"""This code is just basic, things i need to do are:
1. create a place holder for list of items, if no place holder is created then it might impact future cleaning
2. Add more cleaning steps for dataframe"""
def flatten_outer (data):
    full_list = []
    def magic_json (min_data,high_level_key='',inx=0,tot_len=0):
        for k,v in min_data.items():
            fin_key = high_level_key+'.'+k if high_level_key !='' else k
            if isinstance(v, dict): 
                magic_json(v, fin_key)
                
            elif isinstance(v, list):
                for i in range(0, len(v)): 
                    if (isinstance(v[i], dict)):
                        magic_json(v[i], fin_key,inx=i, tot_len=len(v))
                    else: 
                        my_list[fin_key] = v
            else:

                if fin_key in my_list.keys():
                    #print("fin_key: ",fin_key)
                    pl_hd = my_list[fin_key]
                    pl_hd[inx] = v
                    my_list[fin_key] = pl_hd
                else:
                    if inx == 0:
                        #print("inx is 0")
                        if tot_len == 0:
                            my_list[fin_key] = v
                        else:
                            pl_hd = [None]*tot_len
                            pl_hd[0] = v
                            my_list[fin_key] = pl_hd
                    else:
                        print('inx is not 0')
                        dif = tot_len - inx - 1
                        pl_hd = [None] * inx
                        pl_hd.append(v)
                        pl_hd = pl_hd + [None] * dif
                        my_list[fin_key] = pl_hd
                print('finkey is:', fin_key)
                print("finkey value is:", my_list[fin_key])
         
        return my_list
        
    for row in data:
        #print("each row: ",row)
        my_list = dict() 
        cv =  magic_json(row)
        full_list.append(cv)
    
    #dataframe = df_create_clean(full_list)
    
    return full_list

In [84]:
def df_create_clean(full_list):
    df = pd.DataFrame(full_list)
    df = df.where(pd.notnull(df),None)
    cols = df.columns
    for col in cols:
        df[col] = df[col].apply(lambda x: None if (isinstance(x,list) and len(x)==0) else x)
        df[col] = df[col].apply(lambda x: x[0] if (isinstance(x,list) and len(x)==1) else x)
    return df


In [91]:
data = [{'state': 'Florida',
          'shortname': 'FL',
          'info': {
               'governor': 'Rick Scott'
          },
          'counties': [{'name': 'Dade', 'population': 12345,'check':'YesYes'},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
         {'state': 'Ohio',
          'shortname': 'OH',
          'info': {
               'governor': 'John Kasich'
          },
          'counties': [{'name': 'Summit', 'population': 1234},
                       {'name': 'Cuyahoga', 'population': 1337},
                      {'name': 'Cuyahoga2', 'population': 1331,'check':'YesYes'}]}]

In [92]:
ls = flatten_outer(data)
f_df = df_create_clean(ls)
    

finkey is: state
finkey value is: Florida
finkey is: shortname
finkey value is: FL
finkey is: info.governor
finkey value is: Rick Scott
finkey is: counties.name
finkey value is: ['Dade', None, None]
finkey is: counties.population
finkey value is: [12345, None, None]
finkey is: counties.check
finkey value is: ['YesYes', None, None]
finkey is: counties.name
finkey value is: ['Dade', 'Broward', None]
finkey is: counties.population
finkey value is: [12345, 40000, None]
finkey is: counties.name
finkey value is: ['Dade', 'Broward', 'Palm Beach']
finkey is: counties.population
finkey value is: [12345, 40000, 60000]
finkey is: state
finkey value is: Ohio
finkey is: shortname
finkey value is: OH
finkey is: info.governor
finkey value is: John Kasich
finkey is: counties.name
finkey value is: ['Summit', None, None]
finkey is: counties.population
finkey value is: [1234, None, None]
finkey is: counties.name
finkey value is: ['Summit', 'Cuyahoga', None]
finkey is: counties.population
finkey value is:

In [93]:
f_df

,state,shortname,info.governor,counties.name,counties.population,counties.check
0,Florida,FL,Rick Scott,"[Dade, Broward, Palm Beach]","[12345, 40000, 60000]","[YesYes, None, None]"
1,Ohio,OH,John Kasich,"[Summit, Cuyahoga, Cuyahoga2]","[1234, 1337, 1331]","[None, None, YesYes]"
